In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_anthropic import ChatAnthropic
from langchain_groq import ChatGroq
import operator

In [2]:
generator_llm = ChatOpenAI(model = 'gpt-4o')
evaluator_llm = ChatGroq(model = 'llama-3.3-70b-versatile')
optimizer_llm = ChatAnthropic(model = 'claude-sonnet-4-20250514')

In [3]:
from pydantic import BaseModel, Field

class TweetEvaluatioN(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result")
    feedback: str =Field(..., description="feedback for the tweet")

In [4]:
structured_evaluator = evaluator_llm.with_structured_output(TweetEvaluatioN)

In [5]:
class TweetState(TypedDict):
    topic:str
    tweet:str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration:int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [6]:
def generate_tweet(state:TweetState):
    #prompt
    messages =[
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
        Write a short, original and hilarious tweet on the topic: "{state['topic']}"
        Rules:
            - Do NOT use question-answer format
            - Max 280 characters.
            - Use observational humor, irony, sarcasm, or cultural references.
            - Think in meme logic, punchlines, or relatable takes.
            - Use simple, day to day english.""")
    ]

    response = generator_llm.invoke(messages).content

    return {'tweet':response, 'tweet_history':[response]}

In [ ]:
def evaluate_tweet(state:TweetState):

    #prompt
    messages = [
        SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality and tweet format"),
        HumanMessage(content=f"""
        Evaluate the following tweet:
                Tweet: "{state['tweet']}"
                Use the below criteria to evaluate:
                - Originality: Is this fresh, or have you seen it a hundred times before?
                - Humor: Did it genuinely make you smile, laugh or chuckle?
                - Punchiness: Is it short, sharp, and scroll-stopping?
                - Virality Potential - Would people retweet or share it?
                - Format - Is it a well-formed tweet(not a setup-punchline joke, not a Q&A joke, and under 280 characters?)
                
                Auto Reject if :
                - It's written in question-answer format(e.g., "Why did..." or "What happens when...")
                -It exceeds 280 characters
                -It reads lke a traditional setup-punchline joke
                - Don't end with generic, throwaway or deflating lines that weaken the humor(e.g., "Masterpieces of the auntie-uncle universe or vague summaries)
                
                ###Respond ONLY in structured format:
                -evaluation: "approved" or "needs_improvement" 
                -feedback: Two lines explaining the strength and weakness
                """)
    ]

    response = structured_evaluator.invoke(messages)
    return {'evaluation':response.evaluation, 'feedback':response.feedback, 'feedback_history':[response.feedback]}